# Set Up Ais.Net Receiver

In [ ]:
#!connect client --kernel-name clientside

Kernel added: #!clientside

In [ ]:
#i "nuget:C:\Temp\NuGet,*-*"

#r "nuget:Ais.Net.Receiver"
#r "nuget:Ais.Net.Models"

Restore sources C:\Temp\NuGet,*-*

Installed package Ais.Net.Models version 0.1.4

Installed package Ais.Net.Receiver version 0.1.4

In [ ]:
using Ais.Net.Models;
using Ais.Net.Models.Abstractions;
using Ais.Net.Receiver.Configuration;
using Ais.Net.Receiver.Receiver;

static ReceiverHost CreateReceiverHost() =>
    new(new AisConfig { Host = "153.44.253.27", Port = 5631, RetryAttempts = 100, RetryPeriodicity = TimeSpan.Parse("00:00:00:00.500") });

# Set up Bing Maps

In [ ]:
interactive.registerCommandHandler({commandType: 'VesselPositionCommand', handle: c => {
    UpdateVesselPosition(c.command);
}});

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title></title>
    <meta charset="utf-8" />
    <script type='text/javascript'
            src='http://www.bing.com/api/maps/mapcontrol?callback=GetMap' 
            async defer></script>
    <script type='text/javascript'>
        var map;

        function UpdateVesselPosition(position) {

            var pin = getPushpinById(position.mmsi);
            var loc = new Microsoft.Maps.Location(position.lat, position.lon);

            if (pin == null)
            {
                var pin = new Microsoft.Maps.Pushpin(loc, {
                                icon: createRedArrow(position.courseOverGroundDegrees),
                                title: position.name,
                                subTitle: position.mmsi
                            });
                            
                pin.metadata = {
                    id: position.mmsi
                };

                map.entities.push(pin);
            }
            else
            {
                pin.setLocation(loc);
            }
        }

        function GetMap()
        {
            map = new Microsoft.Maps.Map('#myMap', {
                credentials: "AgNSpK5BO4jojYitPjEyli5Z-fZfo-Uvx3P5MXg2Ld_CkcjKft2PwfnPVBTR17uP"
            });

            map.setView({ center: map.getCenter(), zoom: 3 });
        }

        function createRedArrow(heading) {
            var c = document.createElement('canvas');
            c.width = 24;
            c.height = 24;

            var ctx = c.getContext('2d');
        
            //Offset the canvas such that we will rotate around the center of our arrow
            ctx.translate(c.width * 0.5, c.height * 0.5);

            //Rotate the canvas by the desired heading
            ctx.rotate(heading * Math.PI / 180);

            //Return the canvas offset back to it's original position
            ctx.translate(-c.width * 0.5, -c.height * 0.5);
        
            ctx.fillStyle = '#f00';

            //Draw a path in the shape of an arrow.
            ctx.beginPath();
            ctx.moveTo(12, 0);
            ctx.lineTo(5, 20);
            ctx.lineTo(12, 15);
            ctx.lineTo(19, 20);
            ctx.lineTo(12, 0);
            ctx.closePath();
            ctx.fill();
            ctx.stroke();

            //Generate the base64 image URL from the canvas.
            return c.toDataURL();
        }

        function getPushpinById(id) 
        {
            console.log("find " + id);
            var pin;
            for (i = 0; i < map.entities.getLength(); i++) {
                pin = map.entities.get(i);

                if(pin.metadata && pin.metadata.id === id){
                    return pin;
                }
            }
        }
    </script>
    <style>
        #myMap {
            position: relative;
            width: 100%;
            height: 800px;
        }
    </style>
</head>
<body>
    <div id="myMap"></div>
</body>
</html>

<!DOCTYPE html>

# Set up Reaqtor

In [ ]:
#r "bin/Debug/net5.0/Reaqtor.Shebang.App.dll"

### Utilities to work with query engines

In [ ]:
using System.IO;

using Reaqtor.Shebang.App;
using Reaqtor.Shebang.Client;
using Reaqtor.Shebang.Linq;
using Reaqtor.Shebang.Service;

async Task WithNewEngine(IQueryEngineStateStore store, IIngressEgressManager iemgr, Func<ClientContext, Task> action)
{
    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Creating engine...");

    var qe = await QueryEngineFactory.CreateNewAsync(store, ingressEgressManager: iemgr);
    var ctx = qe.Client;

    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Invoking user code...");

    await action(ctx);

    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Checkpointing engine...");

    await qe.CheckpointAsync();

    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Unloading engine...");

    await qe.UnloadAsync();

    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Engine unloaded.");
}

async Task WithExistingEngine(IQueryEngineStateStore store, IIngressEgressManager iemgr, Func<ClientContext, Task> action)
{
    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Recovering engine...");

    var qe = await QueryEngineFactory.RecoverAsync(store, ingressEgressManager: iemgr);
    var ctx = qe.Client;

    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Invoking user code...");

    await action(ctx);

    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Checkpointing engine...");

    await qe.CheckpointAsync();

    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Unloading engine...");

    await qe.UnloadAsync();

    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Engine unloaded.");
}

string SaveStore(InMemoryKeyValueStore store)
{
    var file = Path.Combine(Environment.CurrentDirectory, $"store_{Environment.TickCount64}.txt");

    store.Save(file);

    Console.WriteLine($"Stored dumped to {file}. Size = {new FileInfo(file).Length} bytes");

    return file;
}

### (Optional) Test the setup

In [ ]:
var store = new InMemoryKeyValueStore();
var iemgr = new IngressEgressManager();

await WithNewEngine(store, iemgr, async ctx =>
{
    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Create trivial subscription to test Reaqtor setup.");

    await ctx.SimpleTimer(TimeSpan.FromSeconds(1)).Select((_, i) => i.ToString()).OfType<string>().Cast<string>().SubscribeAsync(ctx.ConsoleOut, new Uri("test://sub/" + Environment.TickCount64));

    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Wait a bit to see events flowing.");

    await Task.Delay(TimeSpan.FromSeconds(5));

    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Done.");
});

2021-03-10 22:17:08 Creating engine...


2021-03-10 22:17:10 Invoking user code...


2021-03-10 22:17:10 Create trivial subscription to test Reaqtor setup.


OnNext(0)


2021-03-10 22:17:10 Wait a bit to see events flowing.


...

OnNext(5)


2021-03-10 22:17:15 Done.


2021-03-10 22:17:15 Checkpointing engine...


2021-03-10 22:17:15 Unloading engine...


2021-03-10 22:17:15 Engine unloaded.


In [ ]:
string file = SaveStore(store);

Stored dumped to d:\Projects\Reaqtive\reaqtor\Reaqtor\Samples\Shebang\Reaqtor.Shebang.App\store_1638088937.txt. Size = 245433 bytes


In [ ]:
var store = InMemoryKeyValueStore.Load(@"d:\Projects\Reaqtive\reaqtor\Reaqtor\Samples\Shebang\Reaqtor.Shebang.App\store_1638088937.txt");
var iemgr = new IngressEgressManager();

await WithExistingEngine(store, iemgr, async ctx =>
{
    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Wait a bit to see events flowing. (NB: There are known issues with Notebooks infra that prevent console output here.)");

    await Task.Delay(TimeSpan.FromSeconds(5));

    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Done.");
});

2021-03-10 22:18:05 Recovering engine...


OnNext(6)


2021-03-10 22:18:07 Invoking user code...


2021-03-10 22:18:07 Wait a bit to see events flowing. (NB: There are known issues with Notebooks infra that prevent console output here.)


OnNext(7)


...

OnNext(11)


2021-03-10 22:18:12 Done.


2021-03-10 22:18:12 Checkpointing engine...


2021-03-10 22:18:12 Unloading engine...


2021-03-10 22:18:12 Engine unloaded.


## Bridge Reaqtor with Ais and kernel

### Get the client kernel

We'll use the client-side kernel object later to send events through an observer abstraction.

In [ ]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Commands;

var clientSideKernel = Kernel.GetKernel("clientside");

### Define data model types

The Reaqtor configuration provided by the Shebang sample stack uses the Nuqleon Data Model to represent entities. While the query operators are functional with arbitrary event types, checkpointing requires the ability to serialize and deserialize events, e.g. if they're stored in the state of `CombineLatest` (to represent the latest values received on the different inputs). The current AIS data types use interfaces, which cannot be deserialized (because the serializer doesn't know which concrete implementation to instantiate).

**Note:** Serialization is pluggable in the Reaqtor query engine, so one could come up with a sophisticated serializer mechanism that supports interfaces and knows which concrete type to instantiate upon deserialization. This could allow AIS events to still be represented using interface implementations that are simple wrappers around underlying state (with minimal allocation overhead, using `Span<T>` etc. underneath), but a second implementation would have to be added to support deserialization (coming from a different underlying representation, e.g. JSON). Note that the Nuqleon Data Model has the advantage that once the queries are formulated using it, these queries and their state are completely portable across machines, and events are exchangable across machines, without having to rely on assemblies or types being deployed. The structural definition of these entity types is captured in checkpoint state (rather than a nominal definition, i.e. referring to a type in an assembly, by name).

In [ ]:
using Nuqleon.DataModel;

public class VesselName
{
    [Mapping("ais://vessel_name/name")]
    public string Name { get; set; }
}

public class VesselNavigation
{
    [Mapping("ais://vessel_nav/position")]
    public Coordinates Position { get; set; }

    [Mapping("ais://vessel_nav/cogdegrees")]
    public float? CourseOverGroundDegrees { get; set; }
}

public class Coordinates
{
    [Mapping("ais://coordinates/lat")]
    public double Latitude { get; set; }

    [Mapping("ais://coordinates/long")]
    public double Longitude { get; set; }
}

public enum MessageType
{
    [Mapping("ais://messagetype/unknown")]
    Unknown,

    [Mapping("ais://messagetype/name")]
    Name,

    [Mapping("ais://messagetype/navigation")]
    Navigation,
}

public class AisMessage
{
    [Mapping("ais://msg/type")]
    public MessageType Type { get; set; }

    [Mapping("ais://msg/name")]
    public VesselName Name { get; set; }

    [Mapping("ais://msg/nav")]
    public VesselNavigation Navigation { get; set; }

    [Mapping("ais://msg/mmsi")]
    public uint Mmsi { get; set; }
}

public class VesselPosition
{
    [Mapping("reaqtor://demo/vessel_tracking/lon")]
    public double Lon { get; set; }

    [Mapping("reaqtor://demo/vessel_tracking/lat")]
    public double Lat { get; set; }

    [Mapping("reaqtor://demo/vessel_tracking/cogdegrees")]
    public float CourseOverGroundDegrees { get; set; }

    [Mapping("reaqtor://demo/vessel_tracking/mmsi")]
    public string Mmsi { get; set; }

    [Mapping("reaqtor://demo/vessel_tracking/name")]
    public string Name { get; set; }
}


### Define the `VesselPositionCommand`

Goal is to have an `IObserver<KernelCommand>` that calls the kernel with the given command instance. Note that the properties in this object are accessed on the JavaScript side in the client-side kernel, so property names should not be altered.

In [ ]:
public class VesselPositionCommand : KernelCommand
{
    public double Lon { get; set; }
    public double Lat { get; set; }
    public float CourseOverGroundDegrees { get; set; }
    public string Mmsi { get; set; }
    public string Name { get; set; }
}

Kernel.Current.RegisterCommandType<VesselPositionCommand>();

### Build a classic `IObserver<T>` for kernel commands

The use of an observer will make it easier to write an Rx query that connects the egress subject coming out of Reaqtor to the client-side kernel.

In [ ]:
class KernelCommandObserver : IObserver<KernelCommand>
{
    private readonly Kernel _kernel;

    public KernelCommandObserver(Kernel kernel) => _kernel = kernel;

    public async void OnNext(KernelCommand value)
    {
        // REVIEW: Fire-and-forget; never checks the result; doesn't support cancellation.
        _ = await _kernel.SendAsync(value);
    }

    public void OnError(Exception error) {}
    public void OnCompleted() {}
}

### Define ingress and egress

We want to send `IAisMessage`s into the query running in the engine, and receive `VesselPositionCommand`s out of the query. To do so, we'll allocate subjects in the `IngressEgressManager`. These will be wired up later:

* inside the query engine by simply writing a query expression that uses `GetIngress<T>(string)` and `GetEgress<T>(string)`;
* outside the query engine by using Rx queries that connect the AIS receiver to the ingress stream, and the egress stream to the kernel command observer.

In [ ]:
var iemgr = new IngressEgressManager();

var ais_data = "aisdata";
var vessel_positions = "vesselpositions";

var ais_publisher = iemgr.CreateSubject<AisMessage>(ais_data);
var vessel_position_consumer = iemgr.CreateSubject<VesselPosition>(vessel_positions);

### Add a helper to collect statistics

In order to keep track of the number of events received by the engine and emitted by the engine, we'll define a little helper. We'll do the counting in `Do` operators in the queries on the outside of the query engine, thus counting what goes in and what comes out of the query running in the engine.

In [ ]:
using System.Threading;

class Stats
{
    private volatile int _rx;
    private volatile int _tx;
    private volatile int _groups;

    public int ReceiveCount => _rx;
    public int SendCount => _tx;
    public int GroupCount => _groups;

    public void OnReceived() => Interlocked.Increment(ref _rx);
    public void OnSend() => Interlocked.Increment(ref _tx);
    public void OnNewGroup() => Interlocked.Increment(ref _groups);

    public void Reset() => (_rx, _tx, _groups) = (0, 0, 0);

    public override string ToString() => $"Received = {ReceiveCount}  Groups = {GroupCount}  Sent = {SendCount}";
}

var stats = new Stats();

### Hook up the input side to receive AIS data

Events don't flow until we call `receiverHost.StartAsync`, so we wrap the whole setup in a helper method we'll call later. First, we wire the `receiverHost.Messages` observable to the `ais_publisher`. To do so, we need to introduce sequence numbers because we're feeding into an `IReliableObserver<IAisMessage>`. We'll just generate increasing numbers obtained using `ToFileTime()` (and assume monotonicity, i.e. nobody is changing the system clock). A real implementation that consumes external events would support replay from the external producer here (e.g. EventHub).

In [ ]:
using System.Reactive.Disposables;
using System.Reactive.Linq;

static IObservable<AisMessage> ConvertToDataModel(this IObservable<IAisMessage> source)
{
    return from msg in source
           where msg is (IVesselName or IVesselNavigation)
           select msg switch
           {
               IVesselName n =>
                  new AisMessage
                  {
                      Type = MessageType.Name,
                      Mmsi = msg.Mmsi,
                      Name = new VesselName()
                      {
                          Name = n.VesselName
                      }
                  },
               IVesselNavigation n =>
                  new AisMessage
                  {
                      Type = MessageType.Navigation,
                      Mmsi = msg.Mmsi,
                      Navigation = new VesselNavigation()
                      {
                          CourseOverGroundDegrees = n.CourseOverGroundDegrees,
                          Position = n.Position switch
                          {
                              Position p => new Coordinates() { Latitude = p.Latitude, Longitude = p.Longitude },
                              null => null
                          }
                      }
                  },
               _ => null
           };
}

IDisposable ReceiveEventsFromAis()
{
    var receiverHost = CreateReceiverHost();

    var ais_publisher_subscription =
        receiverHost.Messages
            .Do(_ => stats.OnReceived())
            .ConvertToDataModel()
            .Select(msg => (sequenceId: (long)DateTimeOffset.UtcNow.ToFileTime(), item: msg))
            .Subscribe(ais_publisher);

    var group_count_subscription =
        receiverHost.Messages
            .GroupBy(msg => msg.Mmsi)
            .Do(_ => stats.OnNewGroup())
            .Subscribe(_ => {});

    var cancel = new CancellationDisposable();

    var t = receiverHost.StartAsync(cancel.Token);

    return StableCompositeDisposable.Create(
        cancel,
        Disposable.Create(() =>
        {
            // NB: Wait for completion after cancellation.
            try { t.GetAwaiter().GetResult(); }
            catch (OperationCanceledException) {}
        }),
        ais_publisher_subscription,
        group_count_subscription);
}

### Hook up the output side to send client kernel commands

To send commands to the kernel, we'll use our `KernelCommandObserver` and hook it up to the `vessel_position_consumer`. This time we need to shake off sequence numbers emitted by the query engine.

In [ ]:
using System.Reactive.Concurrency;

IDisposable SendVesselPositionsToClientKernel()
{
    //
    // NB: The use of the `NewThreadScheduler` is to work around some Notebook threading woes. Every time we
    //     run the helper method here, it will cause new thread creation and establish a proper client kernel
    //     connection. We'll use this helper method within a single cell later.
    //

    var clientSideKernel = Kernel.GetKernel("clientside");

    return vessel_position_consumer
        .Do(_ => stats.OnSend())
        .Select(t => new VesselPositionCommand { Mmsi = t.item.Mmsi, Name = t.item.Name, Lat = t.item.Lat, Lon = t.item.Lon, CourseOverGroundDegrees = t.item.CourseOverGroundDegrees })
        .ObserveOn(NewThreadScheduler.Default)
        .Subscribe(new KernelCommandObserver(clientSideKernel));
}

### Run the query in Reaqtor

Finally, we can run the query in Reaqtor. We'll create the query first, checkpoint the engine, and shut it down. While we could immediately start to process events after creating the subscription, we'll take a first checkpoint to allow for exploration of the state store prior to flowing events.

In [ ]:
using System.Diagnostics;

var subUri = new Uri("test://sub/" + Environment.TickCount64);

var store = new InMemoryKeyValueStore();

await WithNewEngine(store, iemgr, async ctx =>
{
    var source = ctx.GetIngress<AisMessage>(ais_data);
    var sink = ctx.GetEgress<VesselPosition>(vessel_positions);

    var byVessel = source.GroupBy(m => m.Mmsi);

    var vesselNavigationWithNameStream =
        from perVesselMessages in byVessel
        let vesselNavigationUpdates =
            from msg in perVesselMessages
            where msg.Type == MessageType.Navigation
            select msg.Navigation
        let vesselNames =
            from msg in perVesselMessages
            where msg.Type == MessageType.Name
            select msg.Name
        let vesselLocationsWithNames = vesselNavigationUpdates.CombineLatest(vesselNames, (navigation, name) => new { navigation, name })
        from vesselLocationAndName in vesselLocationsWithNames
        select
            new VesselPosition
            {
                Lat = vesselLocationAndName.navigation.Position.Latitude,
                Lon = vesselLocationAndName.navigation.Position.Longitude,
                CourseOverGroundDegrees = vesselLocationAndName.navigation.CourseOverGroundDegrees ?? 0,
                Mmsi = perVesselMessages.Key.ToString(),
                Name = vesselLocationAndName.name.Name.CleanVesselName()
            };

    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Creating subscription...");

    await vesselNavigationWithNameStream.SubscribeAsync(sink, subUri);

    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Shutting down...");
});

2021-03-10 22:19:21 Creating engine...


2021-03-10 22:19:23 Invoking user code...


2021-03-10 22:19:23 Creating subscription...


2021-03-10 22:19:23 Shutting down...


2021-03-10 22:19:23 Checkpointing engine...


2021-03-10 22:19:23 Unloading engine...


2021-03-10 22:19:23 Engine unloaded.


In [ ]:
SaveStore(store);

Stored dumped to d:\Projects\Reaqtive\reaqtor\Reaqtor\Samples\Shebang\Reaqtor.Shebang.App\store_1638224984.txt. Size = 252519 bytes


In [ ]:
using System.Diagnostics;

await WithExistingEngine(store, iemgr, async ctx =>
{
    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Connecting egress to client kernel...");

    using var receiverSubscription = SendVesselPositionsToClientKernel();

    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Starting AIS event receiver...");

    using var publisherSubscription = ReceiveEventsFromAis();

    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Events are being processed...");

    var sw = Stopwatch.StartNew();

    while (sw.Elapsed < TimeSpan.FromSeconds(10))
    {
        await Task.Delay(TimeSpan.FromSeconds(1));

        Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} {stats}");
    }

    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Shutting down...");
});

2021-03-10 22:20:00 Recovering engine...


2021-03-10 22:20:00 Invoking user code...


2021-03-10 22:20:00 Connecting egress to client kernel...


2021-03-10 22:20:00 Starting AIS event receiver...


2021-03-10 22:20:00 Events are being processed...


...

2021-03-10 22:20:11 Received = 345  Groups = 335  Sent = 3


2021-03-10 22:20:11 Shutting down...


2021-03-10 22:20:11 Checkpointing engine...


2021-03-10 22:20:23 Unloading engine...


2021-03-10 22:20:23 Engine unloaded.


In [ ]:
var file = SaveStore(store);

Stored dumped to d:\Projects\Reaqtive\reaqtor\Reaqtor\Samples\Shebang\Reaqtor.Shebang.App\store_1638280093.txt. Size = 6155657 bytes


### Recover the engine from state

We'll reload the store from the file on disk, and then recover the engine from it.

In [ ]:
var store = InMemoryKeyValueStore.Load(file);

In [ ]:
using System.Diagnostics;

stats.Reset();

await WithExistingEngine(store, iemgr, async ctx =>
{
    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Connecting egress to client kernel...");

    using var receiverSubscription = SendVesselPositionsToClientKernel();

    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Starting AIS event receiver...");

    using var publisherSubscription = ReceiveEventsFromAis();

    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Events are being processed...");

    var sw = Stopwatch.StartNew();

    while (sw.Elapsed < TimeSpan.FromSeconds(10))
    {
        await Task.Delay(TimeSpan.FromSeconds(1));

        Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} {stats}");
    }

    Console.WriteLine($"{DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")} Shutting down...");
});

2021-03-10 22:20:53 Recovering engine...


2021-03-10 22:21:07 Invoking user code...


2021-03-10 22:21:07 Connecting egress to client kernel...


2021-03-10 22:21:07 Starting AIS event receiver...


2021-03-10 22:21:07 Events are being processed...


...

2021-03-10 22:21:18 Received = 288  Groups = 280  Sent = 8


2021-03-10 22:21:18 Shutting down...


2021-03-10 22:21:18 Checkpointing engine...


2021-03-10 22:21:45 Unloading engine...


2021-03-10 22:21:45 Engine unloaded.


In [ ]:
SaveStore(store);

Stored dumped to d:\Projects\Reaqtive\reaqtor\Reaqtor\Samples\Shebang\Reaqtor.Shebang.App\store_1638359234.txt. Size = 16743670 bytes
